In [753]:
import os
import glob
from dotenv import load_dotenv
import json
import gradio as gr
from openai import OpenAI

In [754]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [755]:
MODEL_GPT = "gpt-4o-mini"
MODEL_QWEN3b = 'qwen2.5:3b'
db_name = "..\\vector_db"

In [756]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

In [757]:
folders = glob.glob("../knowledge-base/*")

text_loader_kwargs = {'encoding': 'utf-8'}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [758]:
system_message = \
"""
Bạn sẽ đưa ra các câu trả lời hoàn toàn bằng tiếng việt.\
Bạn là một chuyên gia tư vấn về quy chế đào tạo \
cho một đại học ở Việt Nam, Đại học Bách khoa Hà Nội. \
Bạn cần tư vấn chính xác những gì bạn biết và trả lời thành thật những nội dung trong tài liệu bạn được cung cấp. \
Khi nội dung được hỏi không có thông tin trong tài liệu được cung cấp, hãy nói không có thông tin trong tài liệu."""

In [759]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 4436, which is longer than the specified 1000
Created a chunk of size 1205, which is longer than the specified 1000


In [760]:
embeddings = OpenAIEmbeddings()

In [761]:
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

In [762]:
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore: {vectorstore._collection.count()} documents")

Vectorstore: 112 documents


In [763]:
collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 1,536 dimensions


In [764]:
result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
doc_types = [metadata['doc_type'] for metadata in result['metadatas']]
colors = [['blue', 'green'][['document', 'question answer'].index(t)] for t in doc_types]

In [765]:
tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [766]:
from langchain_core.tools import tool
import json
from langchain.agents import initialize_agent

ranking = { 0.0: "Yếu", 2.0: "Trung bình", 2.5: "Khá", 3.2: "Giỏi", 3.6: "Xuất sắc"}

@tool
def get_ranking(grade):
    """
    Trả về xếp loại dựa vào số điểm CPA

    Args:
        grade (float): Số điểm CPA của sinh viên
    """
    grade = grade.replace("'", "\"")

    grade = json.loads(grade).get('grade')
    grade = float(grade)

    if grade > 4 or grade < 0:
        return "Số điểm không hợp lệ trên thang điểm 4"

    for key in sorted(ranking.keys(), reverse= True):
        if grade >= key:
            return ranking[key]

    return "Xuất sắc" 


In [767]:
from langchain_community.chat_models import ChatOllama

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    api_key="ollama",
    model="qwen2.5",
    base_url="http://localhost:11434/v1",
)

tools = [get_ranking]

llm = llm.bind_tools(tools)
agent = initialize_agent(tools, llm, agent_type="zero-shot-react-description", verbose=True)

def handle_get_ranking_tool_call(tool_call):
    arguments = tool_call.get('args')
    grade = arguments.get('grade')
    ranking = agent.run(f"Lấy xếp loại của sinh viên dựa vào điểm số CPA cung cấp: {grade}")
    return ranking


In [768]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
retriever = vectorstore.as_retriever()
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [769]:

def chat(message, history):
    response = llm.invoke(message)
    if response.tool_calls:
        for tool_call in response.tool_calls:
            if tool_call.get('name') == 'get_ranking':
                result = handle_get_ranking_tool_call(response.tool_calls[0])
        return result
    else:
        result = conversation_chain.invoke({"question": message})
        return result["answer"]

In [771]:
view = gr.ChatInterface(chat, type="messages").launch(share = True)

* Running on local URL:  http://127.0.0.1:7885
* Running on public URL: https://a7c3f29ac519058057.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...
Action: get_ranking
Action Input: {"grade": 3.74}
Observation: Xuất sắc
Thought:Final Answer: Sinh viên có điểm số CPA là 3.74 sẽ được xếp loại "Xuất Sắc".

> Finished chain.
